In [1]:
#import libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup 
import requests
import time
import pandas
import random 


def get_brand_item(asin):

    URL = 'https://www.amazon.com/dp/'+ asin

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    product_list = []


    #Brand Name
    brand = soup2.findAll('div',{'class':'a-section a-spacing-none'})

    product_list = list()
    for p_item in brand:
        try:
            p_brand = p_item.find('a')
            #print(p_item.find('a'))
            if p_brand is None:
                continue
            else:
                p_brand = p_item.find('a').text.strip()[7:]
                product_list.append([asin,p_brand])
                #df_asin_brand = pandas.DataFrame(product_list, columns =["asin","brand"])
                #print(pandas.DataFrame(product_list, columns =["asin","brand"]))
            break
        except:
            print("error")

    
    random_number1 = random.randint(2, 5)
    time.sleep(random_number1)
    #df_asin_brand = pandas.DataFrame(product_list, columns =["asin","brand"])
    return(pandas.DataFrame(product_list, columns =["asin","brand"]))





In [2]:
asin = "B08XYYZBQ5"
print(get_brand_item(asin))
#df_asin_brand

Empty DataFrame
Columns: [asin, brand]
Index: []


In [3]:
def get_sellers(asin):    
    
    URL = 'https://www.amazon.com/gp/product/ajax/ref=auto_load_aod?asin='+asin+'&pc=dp&experienceId=aodAjaxMain'

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    offers = []

    pinned_offer = soup2.find('div', {"id":"aod-pinned-offer"})
    soldById = pinned_offer.find('div', {"id":"aod-offer-soldBy"})
    soldByTag = soldById.find('span', {"aria-label":"Amazon.com. Opens a new page"})
    if (soldByTag == None):
        soldByTag = soldById.find('a', {"aria-label":"Opens a new page"})
    soldBy = soldByTag.get_text().strip()
    priceTag = pinned_offer.find('div', {"id":"aod-price-0"}).find('span', {"class":"a-offscreen"})
    price = priceTag.get_text().strip()
    conditionTag = pinned_offer.find('div', {"id":"aod-offer-heading"})
    condition = conditionTag.get_text().strip().replace(" ", "").replace("\n", " ")

    offers.append([soldBy, price, condition])

    for offer in soup2.find_all('div', {"id":"aod-offer"}):
        soldById = offer.find('div', {"id":"aod-offer-soldBy"})
        soldByTag = soldById.find('span', {"aria-label":"Amazon.com. Opens a new page"})
        if (soldByTag == None):
            soldByTag = soldById.find('a', {"aria-label":"Opens a new page"})
        soldBy = soldByTag.get_text().strip()
        priceTag = offer.find('div', {"id":"aod-offer-price"}).find('span', {"class":"a-offscreen"})
        price = priceTag.get_text().strip()
        conditionTag = offer.find('div', {"id":"aod-offer-heading"})
        condition = conditionTag.get_text().strip().replace(" ", "").replace("\n", " ")

        offers.append([soldBy, price, condition])
        
    random_number2 = random.randint(2, 5)
    time.sleep(random_number2)
    return(pandas.DataFrame(offers, columns =["soldBy","price", "condition"]))

    df_sellers = pandas.DataFrame(offers, columns =["soldBy","price", "condition"])

In [4]:
#combine all to a function for 1 listing

def get_excel_one_listing(asin):
        df1= pandas.concat([get_brand_item(asin),get_sellers(asin)],axis=1)
        df2 = df1[["asin","soldBy","brand"]]
        return(df2)
        #df2.to_excel('pandas_to_excel.xlsx', sheet_name='new_sheet_name')

    
    



In [5]:
asin = 'B08L7FXXJ9'
df_001 = get_excel_one_listing(asin)
print(df_001)
type(df_001)

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
asin = 'B07VJ693ZL'
df_002 = get_excel_one_listing(asin)

In [ ]:
#add 2 listings
df_all = pandas.concat([df_001,df_002])
df_all

In [ ]:
df_all3 = pandas.DataFrame()
df_all3 = df_all3.append(df_001)
df_all3.append(df_002)

In [ ]:
df_all.to_excel('pandas_to_excel.xlsx', sheet_name='new_sheet_name')

In [ ]:
#combine more listings

def get_excel_forAll_listings(asin_lists):
    df_all2 = pandas.DataFrame()
    for i in asin_lists:
        if get_excel_one_listing(i) is None:
            continue
        else:
            df_all2 = df_all2.append(get_excel_one_listing(i))
            
    random_number3 = random.randint(2, 5)
    time.sleep(random_number3)      
    return(df_all2.to_excel('pandas_to_excel.xlsx', sheet_name='new_sheet_name'))
        


In [ ]:
asin_lists = ['B08L7FXXJ9','B088WYTXQZ','B09H7S2TXK']
get_excel_forAll_listings(asin_lists)